In [2]:
import torch
import numpy as np
import pandas as pd

In [3]:
class MNISTTrainDataset(torch.utils.data.Dataset):
    def __init__(self, csv_filename, transform=None, target_transform=None):
        self.data = pd.read_csv(csv_filename, skiprows=0) # skip the header
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx):
        label = self.data.iloc[idx, 0]
        image = np.array(list(self.data.iloc[idx,1:]), dtype=np.float32) # all columns but the first
        image = image / image.max() # to set image dynamic range to [0, 1]
        image = image.reshape((28, 28)) # dimension conversion (784) to (28,28)
        if self.transform: # apply an transforms given as input
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
        
    def __len__(self):
        return len(self.data)

In [19]:
full_dataset = MNISTTrainDataset("data/train.csv")

In [21]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(
    full_dataset,
    [train_size, test_size]
)

In [22]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

In [23]:
class DigitNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28 * 28, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = DigitNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [24]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Loss: {running_loss / len(train_loader):.4f}")


Epoch [1/5], Loss: 0.4388
Epoch [2/5], Loss: 0.2074
Epoch [3/5], Loss: 0.1484
Epoch [4/5], Loss: 0.1150
Epoch [5/5], Loss: 0.0918


In [25]:
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test accuracy: {100 * correct / total:.2f}%")

Test accuracy: 96.37%
